In [ ]:
import pathlib

import pandas as pd

import requests
from bs4 import BeautifulSoup

from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder

from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.trainers import BpeTrainer

# Feature Engineering

In our last session, we expanded on the curriculum by looking deeper on how to apply unsupervised learning techniques such as DBScan Clustering and PCA not only for EDA and dimensionality reduction but also on how we can create new features aka feature engineering. 

In this session, we're going to explore Feature Engineering for categorical features and text-based data: preparing you not only in traditional ML but also for Generative AI such as LLMs.

Before we dive into tokenization for LLMs, let’s revisit how we handle feature encoding for categorical data — a crucial step that parallels what LLMs later do implicitly with text.


In [178]:
sentence = "I love everything involving machine learning and the University of California, Berkeley!"

words = sentence.split()

# dict.fromkeys() --> keeps order and the first time each word appears
unique_words = list(dict.fromkeys(words))

df = pd.DataFrame(unique_words, columns=['word'])

## One-hot Encoding

IF you have categorical data with NO inherent order (e.g., colors, countries) → Use one-hot

In [179]:
oh_encoder = OneHotEncoder(categories=[unique_words],sparse_output=False, dtype=int)
encoded_oh = oh_encoder.fit_transform(df[['word']].astype(str))

encoded_df = pd.DataFrame(encoded_oh, columns=[f'is_{word.lower()}' for word in unique_words], index=df.index)

df_one_hot = pd.concat([df.reset_index(drop=True), encoded_df], axis=1)
df_one_hot


,word,is_i,is_love,is_everything,is_involving,is_machine,is_learning,is_and,is_the,is_university,is_of,"is_california,",is_berkeley!
0,I,1,0,0,0,0,0,0,0,0,0,0,0
1,love,0,1,0,0,0,0,0,0,0,0,0,0
2,everything,0,0,1,0,0,0,0,0,0,0,0,0
3,involving,0,0,0,1,0,0,0,0,0,0,0,0
4,machine,0,0,0,0,1,0,0,0,0,0,0,0
5,learning,0,0,0,0,0,1,0,0,0,0,0,0
6,and,0,0,0,0,0,0,1,0,0,0,0,0
7,the,0,0,0,0,0,0,0,1,0,0,0,0
8,University,0,0,0,0,0,0,0,0,1,0,0,0
9,of,0,0,0,0,0,0,0,0,0,1,0,0


## Ordinal Encoding
IF you have categorical data WITH inherent order (e.g., small/medium/large, education levels) → Use ordinal
- Applied to categorical features (words, categories)
- Output is a single integer
- Used for features with inherent order

In [182]:
ord_encoder = OrdinalEncoder(categories=[unique_words],dtype=int)

encoded_ord = ord_encoder.fit_transform(df[['word']].astype(str))

df_ordinal = df.copy()
df_ordinal['ordinal_id'] = encoded_ord.astype(int)

df_ordinal

,word,ordinal_id
0,I,0
1,love,1
2,everything,2
3,involving,3
4,machine,4
5,learning,5
6,and,6
7,the,7
8,University,8
9,of,9


These encoders give numerical structure to non-numeric data. However, they rely entirely on human-defined categories. In contrast, LLMs will learn their own encoding rules.

# LLM Encoding & Features: ML similarities & differences

## PCA v LLM Latent Features
When you apply scaling, PCA, or clustering on numerical data, you're effectively trying to uncover hidden features that aren't visible in your existing features/dataset e.g. PCA explicitly creates **latent features** (principal components) through linear combinations of our existing numerical features. 

In LLMs, latent features are not computed algorithmically or are assumed to be linear - instead they emerge from the raw data as part of training. 

In other words, 
- Traditional ML: **you** *engineer* latent features explicitly --> using tools like PCA or clustering 
- LLMs: the **model** *discovers* latent features implicitly --> through backwardation
  

## Encoding text v features

In traditional ML, we encode text features using techniques such as ordinal encoding or one-hot encoding to transform the features into a numerical format the model can ingest.

Similarly, encoding is used in LLMs to prepare text data prior to model training. The difference is that LLMs begin encoding text data before any features are identified. 
- *LLMs begin by converting raw text into token IDs (numerical form) before the model learns any features or relationships between them.*

That is to say, LLMs identify features implicitly and as part of its training process, not explicitly and as part of the pre-processing process (as we have seen thus far in typical regression and classifier type models)
- *LLMs identify features implicitly within their neural layers during training, rather than explicitly during data preprocessing.*

Summary:
- ML Features: explicit, identified by the Data Scientist before training
- LLM Features: implicit, discovered by the Model during training

As Data Scientists, we still need to prepare the data for encoding, it's just that we're not encoding features.

Let’s visualise how these two approaches — explicit feature engineering and implicit feature discovery — differ at the pipeline level:

<center>

```mermaid

flowchart TD
    %% --- Traditional ML Branch ---
    A["Raw Text Data"] --> B["ML Data Preparation"]
    B --> C["Feature Engineering<br/>(Explicit, Human-Defined)"]
    C --> D["Encoding (Ordinal / One-Hot)"]
    D --> E["Model Training<br/>(Learns relationships between engineered features)"]

    %% --- LLM Branch ---
    A --> F["LLM Data Preparation"]
    F --> G["Chunking<br/>(Split corpus into semantically meaningful text units)"]
    G --> H["Encoding / Tokenisation<br/>(Convert text → token IDs via Tokeniser model)"]
    H --> I["LLM Training<br/>(Learns latent features implicitly within neural layers)"]

    %% --- Tokeniser Details ---
    H --> J["Pre-trained Tokeniser<br/>(e.g., BPE - Byte Pair Encoding)"]
    J --> K["Feature Extraction: identify frequent subword patterns"]
    J --> L["Feature Engineering: map subwords → integer token IDs"]

    %% --- Styling / Grouping ---
    subgraph Traditional_ML["Traditional ML Pipeline"]
        B --> C --> D --> E
    end

    subgraph LLM["LLM Pipeline"]
        F --> G --> H --> I
    end

    style Traditional_ML fill:#f5f5f5,stroke:#666,stroke-width:1px
    style LLM fill:#f5f5f5,stroke:#666,stroke-width:1px
    style A fill:#fff3e0,stroke:#e65100,stroke-width:1px
    style E fill:#e8f5e9,stroke:#2e7d32,stroke-width:1px
    style I fill:#e8f5e9,stroke:#2e7d32,stroke-width:1px
    style J fill:#e3f2fd,stroke:#1565c0,stroke-width:1px
    style K fill:#bbdefb,stroke:#1565c0,stroke-width:1px
    style L fill:#bbdefb,stroke:#1565c0,stroke-width:1px

```

## LLM Data Preparation: Chunking

To help an LLM discover these text features, the first step before training any LLM is to break down their enormous training dataset (corpus) into manageable chunks for encoding and then training.  

Chunks are typically semantically meaningful bodies of text - which can range between an entire document or sentence depending on the training objective. The limitation of the chunk size is called the LLMs context window

After breaking down the corpus into chunks, we then need to break down the chunks into small, unique strings we can then encode.

## Encoding small strings

These small, unique strings are frequently appearing character combinations in languages - often representing words or sub-words. Once identified, these can be mapped to a unique ID and subsequently encoded in the text by replacing the string with the ID. 

In practise, this encoding process is so time-consuming and complicated (how can you guarantee the sub-string and ID matches don't change when you update your training corpus?!?!) that it requires a model of its own. 

As a result, most LLMS use pre-trained encoders created from other models.

These models are referred to as 'Tokenisers'.


## Tokenisers: text encoding models for LLMs

A "Tokenizer" is simply a model that CONSISTENTLY encodes your text data into a numerical format for LLM ingestion. This process is called tokenisation. Under the hood, the most popular algo for tokenisation is BPE.

**Byte Pair Encoding (BPE)** is a popular encoding algo used in tokenisers that seeks to extract and engineer features. 

BPE seeks to:
- identify the most frequent combinations of characters in your text dataset as features → **feature extraction**
- map each feature to a unique integer (token id) suitable for model training → **feature engineering**

Want to visualise text tokenisation in OpenAI models? [Tiktokenizer ](https://tiktokenizer.vercel.app/)

<center>

```mermaid
flowchart TD
    %% --- INPUT STAGE ---
    A["① Raw Text Corpus<br/>(e.g. 'machine learning models are amazing')"] --> B["② Select Tokenization Algorithm<br/>(e.g. BPE, Unigram, WordPiece)"]
    B --> C["③ Initialize Vocabulary<br/>(Each character = token)<br/>Vocabulary size = 256"]

    %% --- FEATURE EXTRACTION STAGE ---
    C --> D["④ Identify Frequent Pairs<br/>(Find most common adjacent characters/subwords)"]
    D --> E["⑤ Merge Frequent Pairs<br/>(Combine 'm' + 'a' → 'ma', add new token)"]
    E --> F["⑥ Update Vocabulary<br/>(Add merged token with new Token ID)"]

    %% --- FEATURE ENGINEERING STAGE ---
    F --> G["⑦ Replace Original Tokens<br/>(Replace 'm' 'a' with 'ma' Token ID)"]
    G --> H["⑧ Repeat Merging Iteratively<br/>Until no frequent pairs remain"]

    %% --- OUTPUT STAGE ---
    H --> I["⑨ Final Vocabulary<br/>(Subwords / Character Pairs + Unique Token IDs)"]

    %% --- STYLING ---
    style A fill:#fff3e0,stroke:#e65100,stroke-width:1px
    style B fill:#ffe0b2,stroke:#f57c00,stroke-width:1px
    style C fill:#fff9c4,stroke:#f9a825,stroke-width:1px
    style D fill:#e3f2fd,stroke:#1565c0,stroke-width:1px
    style E fill:#bbdefb,stroke:#1565c0,stroke-width:1px
    style F fill:#90caf9,stroke:#1565c0,stroke-width:1px
    style G fill:#e8f5e9,stroke:#2e7d32,stroke-width:1px
    style H fill:#c8e6c9,stroke:#2e7d32,stroke-width:1px
    style I fill:#aed581,stroke:#2e7d32,stroke-width:1px

    %% --- GROUP LABELS ---
    subgraph TI["🧩 TOKENIZER INPUTS"]
        A --> B --> C
    end

    subgraph FE["🧠 Feature Extraction Phase"]
        D --> E --> F
    end

    subgraph EN["⚙️ Feature Engineering Phase"]
        G --> H
    end



```

#### 1. Data Collection & Cleaning

In [ ]:
dataset_dir = pathlib.Path("../datasets")
dataset_dir.mkdir(parents=True, exist_ok=True)

url = "https://em-executive.berkeley.edu/professional-certificate-machine-learning-artificial-intelligence"

corpus_source = requests.get(url)
corpus_source.raise_for_status()
soup = BeautifulSoup(corpus_source.text, "html.parser")

In [ ]:
paragraphs = [p.get_text().strip() for p in soup.find_all("p") if p.get_text().strip()]
full_text = "\n".join(paragraphs)

raw_file = dataset_dir / "corpus_full.txt"
with raw_file.open("w", encoding="utf-8") as f:
    f.write(full_text)

### 2: Tokenizer Algorithm
Select the tokenizer algorightm and any pre-processing steps

In [ ]:
tokenizer = Tokenizer(BPE())
tokenizer.pre_tokenizer = Whitespace()

### 3-8: Tokenizer Training

Now that our text corpus is saved, let’s train our own BPE-based tokenizer. This is the same type of process used by models like GPT, except on a massive scale.

In [172]:
trainer = BpeTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"],vocab_size=2000)

tokenizer.train(files=[str(raw_file)], trainer=trainer)

### 9. Final Vocabulary
That's it. You have now created a "vocabulary" containing a token (string): token id for every frequent combination of characters it could find!

Note: as languages increase with words, use cases get more defined (for LLMs), and models become more powerful, I expect vocab sizes to increase to incorporate more whole words e.g. LinkedIn, Professor etc.

Below is a sample of the tokens our tokenizer learned — these represent frequent subword units discovered from the training corpus.

In [173]:
print("Vocab size:", tokenizer.get_vocab_size())

vocab = tokenizer.get_vocab()

for token, idx in list(vocab.items())[:10]:
    print(f"{idx}: {token}")

Vocab size: 1661
900: 04
1073: nal
220: Education
207: ecutive
578: Colle
1293: learners
68: n
703: online
209: port
797: Develop


# Closing Remarks

## Text Encoding comparison: ML v LLM

Let’s now merge our outputs — one-hot, ordinal, and BPE — into a single table to visually compare how traditional and LLM encodings differ.

In [181]:
token_ids = []
for word in unique_words:
    encoded = tokenizer.encode(word)
    token_ids.append(encoded.ids)  # .ids gives the integer token IDs

# --- Build DataFrame ---
df_bpe = pd.DataFrame({
    "word": unique_words,
    "token_id": token_ids
})


df_encode = df_bpe.copy()
df_encode = df_encode.merge(df_ordinal, on='word')
df_encode = df_encode.merge(df_one_hot, on='word')
df_encode 

,word,token_id,ordinal_id,is_i,is_love,is_everything,is_involving,is_machine,is_learning,is_and,is_the,is_university,is_of,"is_california,",is_berkeley!
0,I,[39],0,1,0,0,0,0,0,0,0,0,0,0,0
1,love,"[137, 114]",1,0,1,0,0,0,0,0,0,0,0,0,0
2,everything,"[1024, 79, 86, 98]",2,0,0,1,0,0,0,0,0,0,0,0,0
3,involving,"[83, 76, 722]",3,0,0,0,1,0,0,0,0,0,0,0,0
4,machine,[750],4,0,0,0,0,1,0,0,0,0,0,0,0
5,learning,[448],5,0,0,0,0,0,1,0,0,0,0,0,0
6,and,[96],6,0,0,0,0,0,0,1,0,0,0,0,0
7,the,[99],7,0,0,0,0,0,0,0,1,0,0,0,0
8,University,[842],8,0,0,0,0,0,0,0,0,1,0,0,0
9,of,[103],9,0,0,0,0,0,0,0,0,0,1,0,0


### Lightbulb moment
What is cool about this? In our original sentence, the word "involving" (idx: 3) is tokenised BUT doesn't exist in the corpus!

Instead, the string:
- "inv" appeared 8 times in the corpus from the word "invite" --> encoded with token id: 83
- "olv" appeared 6 times in the corpus, from words such as "solve" and "evolve" --> encoded with token id: 76
- "ing" appeared 152 times in the corpus, from lots of words! --> encoded with token id 722

## Putting it all together: how an LLM encodes during inference

We discussed a lot about feature engineering and encoding today - but this is essentially what happens when every time you run a prompt/query whenever you use an LLM such as ChatGPT!

User Input: "I love everything involving machine learning and the University of California, Berkeley!"

LLM's tokenizer: breaks this text into subword tokens using greedy longest-match-first matching against its  vocabulary, matches it against a token, finds its token ID...(see diagram below for a fuller picture)


**The takeaway:** whether you’re engineering one-hot vectors or training massive LLMs, both rely on feature encoding — the key difference lies in who engineers the features: you, or the model.


Have a great day!

<center>


``` mermaid
flowchart TD
    %% --- User Input ---
    A["<b>① User Input</b><br/>'I love everything involving machine learning...'"]

    %% --- Tokenizer Stage ---
    A --> B["<b>② Tokenizer: Identify Tokens</b><br/>(Greedy Longest-Match-First Search)"]
    B --> C["<b>③ Tokenizer: Vocabulary Lookup</b><br/>(Each Token → Token ID)"]

    %% --- LLM Input Processing ---
    C --> D["④ LLM Input Layer: Embedding Lookup<br/>(Token IDs → Embedding Vectors)"]
    D --> E["⑤ LLM Input Layer: Positional Encoding<br/>(Add Sequence Order Info to Vectors)"]

    %% --- LLM Learning ---
    E --> F["⑥ RUN MODEL!<br/>(to be continued...)"]

    %% --- Today's Lesson ---
    subgraph TL["🧠 Discussed today"]
        A --> B --> C
    end

    %% --- Styling ---
    style TL fill:#fff3e0,stroke:#f57c00,stroke-width:2px,stroke-dasharray: 5 5
    style A fill:#fffde7,stroke:#f9a825,stroke-width:1px
    style B fill:#e3f2fd,stroke:#1565c0,stroke-width:1px
    style C fill:#bbdefb,stroke:#1565c0,stroke-width:1px
    style D fill:#e8f5e9,stroke:#2e7d32,stroke-width:1px
    style E fill:#c8e6c9,stroke:#2e7d32,stroke-width:1px
    style F fill:#f1f8e9,stroke:#2e7d32,stroke-width:1px



```